# Configuration

In [2]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

nRows = 10
nCols = 8
objp = np.zeros((nRows*nCols,3), np.float32)
objp[:,:2] = np.mgrid[0:nCols,0:nRows].T.reshape(-1,2)


# Parameter of left camera

In [ ]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space

left_imgpoints = [] # 2d points in image plane.
left_images = glob.glob('left.jpg')
for fname in left_images:

    img = cv2.imread(fname)
    img = cv2.resize(img, (img.shape[1]//4, img.shape[0]//4))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    size = gray.shape[::-1]
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (nCols,nRows),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        left_imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nCols, nRows), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# 相機校正
# left_ret, left_mtx, left_dist, left_rvecs, left_tvecs = cv2.calibrateCamera(objpoints, imgpoints,size)


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
right_imgpoints = [] # 2d points in image plane.

right_images = glob.glob('right.jpg')

for fname in right_images:
    img = cv2.imread(fname)
    img = cv2.resize(img, (img.shape[1]//4, img.shape[0]//4))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    size = gray.shape[::-1]
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (nCols,nRows),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        right_imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nCols, nRows), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# 相機校正
# right_ret, right_mtx, right_dist, right_rvecs, right_tvecs = cv2.calibrateCamera(objpoints, imgpoints,size, None, None)

ret, left_mtx, left_dist, right_mtx, right_dist, R, T, E, F = cv2.stereoCalibrate(objpoints, left_imgpoints, right_imgpoints, size, None, None, None, None)
print("left_ret: {}".format(ret))
print("left_mtx: {}".format(left_mtx))        # 內部參數矩陣
print("left_dist: {}".format(left_dist))      # 畸變係數   distortion cofficients = (k_1,k_2,p_1,p_2,k_3)
print("left_rvecs: {}".format(left_rvecs))    # 旋转向量  # 外参数
print("left_tvecs: {}".format(left_tvecs))    # 平移向量  # 外参数
print("===================")



cv2.destroyAllWindows()

# Parameter of right camera

In [10]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('right.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    size = gray.shape[::-1]
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (nCols,nRows),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nCols, nRows), corners2,ret)
        img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
        cv2.imshow('img',img)
        cv2.waitKey(0)

# 相機校正
right_ret, right_mtx, right_dist, right_rvecs, right_tvecs = cv2.calibrateCamera(objpoints, imgpoints,size)
 
print("right_ret: {}".format(right_ret))
print("right_mtx: {}".format(right_mtx))        # 內部參數矩陣
print("right_dist: {}".format(right_dist))      # 畸變係數   distortion cofficients = (k_1,k_2,p_1,p_2,k_3)
print("right_rvecs: {}".format(right_rvecs))    # 旋转向量  # 外参数
print("right_tvecs: {}".format(right_tvecs))    # 平移向量  # 外参数
print("===================")
cv2.destroyAllWindows()

TypeError: calibrateCamera() missing required argument 'cameraMatrix' (pos 4)

# Find Fundamental matrix

In [11]:
from matplotlib import pyplot as plt

# left = cv2.imread('left.jpg', 0)
# right = cv2.imread('right.jpg', 0)

left = left_rotated
right = right_rotated

left = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)
right = cv2.cvtColor(right, cv2.COLOR_BGR2GRAY)

sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(left, None)
kp2, des2 = sift.detectAndCompute(right, None)

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i, (m, n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)

# find fundamental matrix
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_LMEDS)

# we select only inlier points
pts1 = pts1[mask.ravel() == 1]
pts2 = pts2[mask.ravel() == 1]

In [12]:
def drawLines(img1, img2, lines, pts1, pts2):
# img1 - image on which we draw the epilines for the points in img2
# lines - correspoding epilines

    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1]])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color, 2)
        img1 = cv2.circle(img1, tuple(pt1), 7, color, 10)
        img2 = cv2.circle(img2, tuple(pt2), 7, color, 10)

    return img1, img2

In [14]:
# Find epilines corresponding to points in right image(second image) and drawing ites line on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img3, img4 = drawLines(left, right, lines1, pts1, pts2)

# Find epilines corrsponding to points in left image(first image) and drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img5, img6 = drawLines(right, left, lines2, pts2, pts1)

# plt.subplot(121), plt.imshow(img3)
# plt.subplot(122), plt.imshow(img5)
# plt.show()

cv2.imshow('img3', img3)
cv2.imshow('img5', img5)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Image Rectification

In [4]:
import math

# r_tvecs = np.array(right_tvecs[0]).flatten()
# l_tvecs = np.array(left_tvecs[0]).flatten()
# rotate_matrix = cv2.Rodrigues(left_rvecs[0])[0]

# choose orthogonal basis
# T = (r_tvecs - l_tvecs)

e1 = T.flatten()
e2 = np.array([-1 * e1[1], e1[0], 0])
e3 = np.cross(e1, e2)

print(T)

# normalize
e1 = e1 / np.linalg.norm(e1, 2)
e2 = e2 / np.linalg.norm(e2, 2)
e3 = e3 / np.linalg.norm(e3, 2)

# assert
assert np.inner(e1, e3) < 1e-10
assert np.inner(e2, e3) < 1e-10
assert np.inner(e1, e2) < 1e-10

rotate_rec = np.array([e1.T, e2.T, e3.T])
print(rotate_rec)


[[6.04965526e+00]
 [2.78275783e+00]
 [8.16008914e-04]]
[[ 9.08495212e-01  4.17895244e-01  1.22542551e-04]
 [-4.17895247e-01  9.08495219e-01  0.00000000e+00]
 [-1.11329322e-04 -5.12099499e-05  9.99999992e-01]]


# Rotate left image 

In [6]:
left = cv2.imread("left.jpg")
left_rotated = cv2.warpPerspective(left, rotate_rec, (left.shape[1], left.shape[0])) 

left = cv2.resize(left, (left.shape[1]//4, left.shape[0]//4))
left_rotated = cv2.resize(left_rotated, (left_rotated.shape[1]//4, left_rotated.shape[0]//4))

cv2.imshow("orgi", left)
cv2.imshow("left_rotated", left_rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Rotate right imgage

In [7]:
right = cv2.imread("right.jpg")
right_rotated = cv2.warpPerspective(right, rotate_rec, (right.shape[1], right.shape[0]))
right_rotated = cv2.warpPerspective(right_rotated, R, (right.shape[1], right.shape[0]))

right = cv2.resize(right, (right.shape[1]//4, right.shape[0]//4))
right_rotated = cv2.resize(right_rotated, (right_rotated.shape[1]//4, right_rotated.shape[0]//4))

cv2.imshow("orgi", right)
cv2.imshow("right_rotated", right_rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
import math
import numpy as np
print(tuple(left_rvecs[0].flatten()))
R = cv2.Rodrigues(left_rvecs[0])
print(R[0])

(-0.06672883341645154, 0.12394726991513878, -0.01836739381351619)
[[ 0.99216303  0.01417727  0.12414316]
 [-0.02243425  0.99760897  0.06536854]
 [-0.12291958 -0.06764131  0.9901088 ]]


In [11]:

a = (0.2,0.4,0.8)
print (a)
R = cv2.Rodrigues(a)
print (R[0])

(0.2, 0.4, 0.8)
[[ 0.62722765 -0.65533377  0.42085997]
 [ 0.72988824  0.6831435  -0.02404381]
 [-0.27175103  0.32226169  0.90680691]]


right_ret: 5.2809885243624946
right_mtx: [[2.87429613e+03 0.00000000e+00 1.38678563e+03]
 [0.00000000e+00 3.59412012e+03 2.43206382e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
right_dist: [[ 0.17008908  0.5594629  -0.11466527  0.03202325 -1.73737707]]
right_rvecs: [array([[ 0.21536264],
       [-0.69502468],
       [-3.16430504]]), array([[ 1.42988455],
       [ 0.78369162],
       [-0.96612497]]), array([[ 0.03491094],
       [-0.7407694 ],
       [-2.82817753]]), array([[ 0.15733505],
       [-0.71284472],
       [-3.16127251]]), array([[-0.30602719],
       [-0.57462676],
       [-1.88024326]]), array([[-0.31468843],
       [-0.40949656],
       [-1.5069855 ]]), array([[-0.28969038],
       [-0.30249604],
       [-1.30432728]]), array([[-0.38160499],
       [-0.40300393],
       [-1.42393631]]), array([[-0.32094819],
       [-0.49639008],
       [-1.65967493]]), array([[ 1.32100755],
       [ 1.04469551],
       [-1.35973864]]), array([[ 0.75415597],
       [-1.30000132],
